In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

## 데이터전처리
* 머신러닝 알고리즘을 익히는 것 못치 않게
  데이터 전처리 역시 중요한 과정 중에 하나
* 무엇보다 머신러닝 알고리즘을 적용하기 전에
   데이터에 대해 미리 처리해야 하는 기본사항이 존재
* `결측치 처리`
  + NaN, Null은 허용되지 않음
* `원핫인코딩` 
  + 머신러닝 알고리즘들은 문자열값을 데이터의 입력값으로 허용하지 않음
  + 따라서, 모든 문자열값은 인코딩해서 숫자형으로 변환해둬야 함
  + 한편, 텍스트 데이터들은 벡터화해서 처리
  + 머신러닝을 위한 인코딩은 `레이블인코딩` 과 `원핫인코딩` 등이 있음

In [2]:
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score

### 레이블 인코딩
* 범주형값을 숫자형값으로 변환함

In [3]:
from sklearn.preprocessing import LabelEncoder

items = ['티비','냉장고','가스렌지','에어콘','컴퓨터']

In [4]:
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)

In [5]:
labels

array([4, 1, 0, 2, 3])

In [6]:
# 인코딩된 순서
encoder.classes_

array(['가스렌지', '냉장고', '에어콘', '컴퓨터', '티비'], dtype='<U4')

## LabelEncoder 시용시 문제점
* 문자열값을 숫자형으로 변환시켰을때 발생할 수 있는 문제는 
  각 값의 대소 관계를 통해 중요도 여부가 존재할 수 있음
* 즉, 인코딩 된 값에 서수척도가 생길수 있음
* 따라서, 대소관계가 있는 데이터를 분석할 경우 정확도에 영향을 미칠수있음
  + => 원핫인코딩을 사용함으로써 문제해결
  
'티비','냉장고','가스렌지','에어콘','컴퓨터'   
1	0	0	0	0   
0	1	0	0	0   
0	0	1	0	0   
0	0	0	1	0   
0	0	0	0	1  

In [7]:
from sklearn.preprocessing import OneHotEncoder

# 먼저, LabelEncoder로 문자열을 숫자값으로 변환해두어야 함!
# 그런 다음, 1차원 데이터를 2차원 데이터 변환
labels = labels.reshape(-1, 1)   # -1 : 행을 열로 변환 (가로->세로)
labels

array([[4],
       [1],
       [0],
       [2],
       [3]])

In [8]:
onehot = OneHotEncoder()
onehot.fit(labels)
ohlabels = onehot.transform(labels)

# ['티비','냉장고','가스렌지','에어콘','컴퓨터']
ohlabels.toarray()

array([[0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]])

### pandas의 원핫인코딩
+ get_dummies 함수 사용
+ 단, 변환대상은 반드시 데이터프레임이어야 함!

In [9]:
df = pd.DataFrame({'':items})
pd.get_dummies(df)

,_가스렌지,_냉장고,_에어콘,_컴퓨터,_티비
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,0,1,0,0
4,0,0,0,1,0


## 특성 스케일링과 표준화/정규화
* 서로 다른 범위, 단위의 변수값을
  일정수준으로 맞추는 작업을 **특성feature 스케일링**이라 함
* 어떤 데이터의 값이 정수와 실수가 혼용되어 있거나
* 값의 범위가 1 ~ 100, 0 ~ 0.001, 1 ~ 10000 등등의 경우
* 데이터 분석시 많은 cpu 파워/메모리가 필요하고
* 학습시 느려질수 있으며 제대로 된 결과가 나오지 않을 수 있음
* 이것을 제대로 변환하는 방법은 `정규화` 와 `표준화` 가 있음

### 표준화
* 특성값을 평균이 0이고 표준편차가 1인 정규분포를 가진값으로 변환하는 것


In [10]:
X = np.arange(9) -3
X

array([-3, -2, -1,  0,  1,  2,  3,  4,  5])

In [11]:
print(np.mean(X, axis=0))
print(np.std(X, axis=0))

1.0
2.581988897471611


In [12]:
X = X.reshape(-1,1) # wide -> Long 변환
X

array([[-3],
       [-2],
       [-1],
       [ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5]])

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)  
XX = scaler.transform(X)
XX

array([[-1.54919334],
       [-1.161895  ],
       [-0.77459667],
       [-0.38729833],
       [ 0.        ],
       [ 0.38729833],
       [ 0.77459667],
       [ 1.161895  ],
       [ 1.54919334]])

In [14]:
print(np.mean(XX, axis=0))
print(np.std(XX, axis=0))

[0.]
[1.]


## 정규화
* 특성값을 최소 0, 최대 1사이의 값으로 변환하는 것
* 데이터의 분포가 정규분포를 따르지 않을때 적용

In [15]:
print(np.min(X, axis=0))
print(np.max(X, axis=0))

[-3]
[5]


In [16]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)  
XXX = scaler.transform(X)
XXX

array([[0.   ],
       [0.125],
       [0.25 ],
       [0.375],
       [0.5  ],
       [0.625],
       [0.75 ],
       [0.875],
       [1.   ]])

In [17]:
print(np.min(XXX, axis=0))
print(np.max(XXX, axis=0))

[0.]
[1.]


## 중앙값, 사분위수 기반 스케일링
* RobustScaler
* 이상치나 극단치에 영향을 덜 받는 스케일링 기법


In [18]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(X)  
XXXX = scaler.transform(X)
XXXX

array([[-1.  ],
       [-0.75],
       [-0.5 ],
       [-0.25],
       [ 0.  ],
       [ 0.25],
       [ 0.5 ],
       [ 0.75],
       [ 1.  ]])

## 스케일링을 적용후 타이타닉 생존여부 분석하기

In [19]:
titanic = pd.read_csv('data/titanic2.csv')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306 entries, 0 to 1305
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1306 non-null   int64  
 1   age       1306 non-null   float64
 2   sibsp     1306 non-null   int64  
 3   parch     1306 non-null   int64  
 4   fare      1306 non-null   float64
 5   gender    1306 non-null   int64  
 6   Embarked  1306 non-null   int64  
 7   Title     1306 non-null   int64  
 8   survived  1306 non-null   int64  
dtypes: float64(2), int64(7)
memory usage: 92.0 KB


### 승객의 나이, 요금을 범주형으로 변환

In [20]:
# 나이 범주 : 5, 10, 20,30, 40, 50, 60, 70,80, 90~
def getAge(x):
    age = 'A'
    if (x >= 5) & (x < 10): age = 'B'
    elif (x >= 10) & (x < 20): age = 'C'
    elif (x >= 20) & (x < 30): age = 'D'
    elif (x >= 30) & (x < 40): age = 'E'
    elif (x >= 40) & (x < 50): age = 'F'
    elif (x >= 50) & (x < 60): age = 'G'
    elif (x >= 60) & (x < 70): age = 'H'
    elif (x >= 70) & (x < 80): age = 'I'                  
    elif (x >= 80) & (x < 90): age = 'J'
    elif x >= 90: age = 'K'
    return age

In [21]:
# 요금 : 100, 200, 300, 400, 500~
def getFare(x):
    fare = 'A'
    if (x >= 100) & (x < 200): fare = 'B'
    elif (x >= 200) & (x < 300): fare = 'C'
    elif (x >= 300) & (x < 400): fare = 'D'
    elif (x >= 400) & (x < 500): fare = 'E'
    elif x >= 500: fare = 'F'
    return fare

In [22]:
titanic['age'] = titanic.age.apply(lambda x: getAge(x))

encoder = LabelEncoder()
encoder.fit(titanic['age'])
titanic['age'] = encoder.transform(titanic['age'])
titanic.head()

,pclass,age,sibsp,parch,fare,gender,Embarked,Title,survived
0,1,3,0,0,211.3375,0,2,10,1
1,1,0,1,2,151.5500,1,2,9,1
2,1,0,1,2,151.5500,0,2,10,0
3,1,4,1,2,151.5500,1,2,13,0
4,1,3,1,2,151.5500,0,2,14,0


In [23]:
titanic['fare'] = titanic.fare.apply(lambda x: getFare(x))

encoder = LabelEncoder()
encoder.fit(titanic['fare'])
titanic['fare'] = encoder.transform(titanic['fare'])
titanic.head()

,pclass,age,sibsp,parch,fare,gender,Embarked,Title,survived
0,1,3,0,0,2,0,2,10,1
1,1,0,1,2,1,1,2,9,1
2,1,0,1,2,1,0,2,10,0
3,1,4,1,2,1,1,2,13,0
4,1,3,1,2,1,0,2,14,0


In [24]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306 entries, 0 to 1305
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   pclass    1306 non-null   int64
 1   age       1306 non-null   int32
 2   sibsp     1306 non-null   int64
 3   parch     1306 non-null   int64
 4   fare      1306 non-null   int32
 5   gender    1306 non-null   int64
 6   Embarked  1306 non-null   int64
 7   Title     1306 non-null   int64
 8   survived  1306 non-null   int64
dtypes: int32(2), int64(7)
memory usage: 81.8 KB


In [42]:
data = titanic.iloc[:,:8]
target = titanic.survived
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306 entries, 0 to 1305
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   pclass    1306 non-null   int64
 1   age       1306 non-null   int32
 2   sibsp     1306 non-null   int64
 3   parch     1306 non-null   int64
 4   fare      1306 non-null   int32
 5   gender    1306 non-null   int64
 6   Embarked  1306 non-null   int64
 7   Title     1306 non-null   int64
dtypes: int32(2), int64(6)
memory usage: 71.5 KB


### 표준화 적용

In [43]:
scaler = StandardScaler()
scaler.fit(data)
data2 = scaler.transform(data)

In [44]:
X_train, X_test, Y_train,Y_test = train_test_split(data2,target, train_size=0.7, stratify=target, random_state=2211161705)

In [45]:
dtclf = DecisionTreeClassifier(random_state=2211161705)

dtclf.fit(X_train, Y_train)
pred = dtclf.predict(X_test)

accuracy_score(Y_test,pred)

0.7602040816326531

In [46]:
rfclf = RandomForestClassifier(random_state=2211161705)

rfclf.fit(X_train, Y_train)
pred = rfclf.predict(X_test)

accuracy_score(Y_test,pred)

0.7346938775510204

In [47]:
scores = cross_val_score(rfclf, data, target, scoring='accuracy', cv=10)

In [48]:
print(np.mean(data2[:, 1], axis=0))
print(np.std(data2[:, 1], axis=0))

-8.704964603492805e-17
1.0


### 정규화 적용

In [49]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(data)
data2 = scaler.transform(data)

In [52]:
X_train, X_test, Y_train,Y_test = train_test_split(data2,target, train_size=0.7, stratify=target, random_state=2211161555)

In [53]:
dtclf = DecisionTreeClassifier(random_state=2211161315)

dtclf.fit(X_train, Y_train)
pred = dtclf.predict(X_test)

accuracy_score(Y_test,pred)

0.7678571428571429

In [54]:
rfclf = RandomForestClassifier()

rfclf.fit(X_train, Y_train)
pred = rfclf.predict(X_test)

accuracy_score(Y_test,pred)

0.7882653061224489

In [55]:
print(np.mean(data2[:, 1], axis=0))
print(np.std(data2[:, 1], axis=0))

0.37706312744597587
0.1572462742173892


### 중앙값, 사분위수 기반 스케일링 적용

In [56]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(data)
data2 = scaler.transform(data)

In [57]:
X_train, X_test, Y_train,Y_test = train_test_split(data2,target, train_size=0.7, stratify=target, random_state=2211161555)

In [58]:
dtclf = DecisionTreeClassifier()

dtclf.fit(X_train, Y_train)
pred = dtclf.predict(X_test)

accuracy_score(Y_test,pred)

0.7704081632653061

In [59]:
rfclf = RandomForestClassifier()

rfclf.fit(X_train, Y_train)
pred = rfclf.predict(X_test)

accuracy_score(Y_test,pred)

0.7882653061224489

In [60]:
print(np.mean(data2[:, 1], axis=0))
print(np.std(data2[:, 1], axis=0))

0.39356814701378257
1.4152164679565031
